In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gc
import time
from itertools import chain
from _thread import start_new_thread

import tensorflow as tf
from keras import backend as K
from keras import regularizers, optimizers

from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization

from numba import cuda
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

# @TODO build CI
import scipy.stats

cuda.select_device(0)

Using TensorFlow backend.


<weakproxy at 0x000001D7F21170E8 to Device at 0x000001D7F1FB8278>

In [2]:
# @TODO Get this working an import

def LabelPicker(label='Cardiomegaly'):
    global train_df
    global test_df
    global df
    df = de_df[de_df['Finding Labels'] == label]
    
    df = pd.concat([df,NoF_df])
    
    train_df = pd.merge(train_filter,df , on='Image Index', how='inner')
    test_df = pd.merge(test_filter,df , on='Image Index', how='inner')
    
def ResetHard():
    #Dump GPU memory after save
    print('Starting Dump')
    cuda.close()
    print('Cuda Closed')
    K.clear_session()
    print('Session Cleared')
    del model
    print('del model')
    gc.collect()
    print('Finishing Dump')
    # My god damn hero
    # https://github.com/keras-team/keras/issues/9379#issuecomment-477414618
    # The reason behind it is: Tensorflow is just allocating memory to the GPU, while CUDA is responsible for managing the GPU memory.
    

#Attach CSV to Images Folder and defline our traiing validation and test
def vaild_gen(batch_fit, Class_Value, TARGET_SIZE):
    global valid_generator
    valid_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image Index",
        y_col=label,
        subset="validation",
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def train_gen(batch_fit, Class_Value, TARGET_SIZE):
    global train_generator
    train_generator=datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image Index",
        y_col=label,
        subset="training",
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def test_gen(batch_fit, Class_Value, TARGET_SIZE):
    global test_generator
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="../images/",
        x_col="Image Index",
        y_col=label,
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def init_gens(batch_fit, class_value, target):
    start_new_thread(vaild_gen,(batch_fit, class_value,target,  ))
    start_new_thread(train_gen,(batch_fit, class_value,target, ))
    start_new_thread(test_gen,(batch_fit, class_value,target, ))

In [3]:
# Load data
BBox_df = pd.read_csv('../BBox_List_2017.csv',dtype=str)
de_df = pd.read_csv('../Data_Entry_2017.csv',dtype=str)
train_filter = pd.read_csv('../train_val_list.txt',dtype=str)
train_filter.columns=['Image Index']
test_filter = pd.read_csv('../test_list.txt',dtype=str)
test_filter.columns=['Image Index']

In [4]:
bad_ages = de_df[de_df['Patient Age'].astype(int) >= 110].index

de_df = de_df.drop(bad_ages)

de_df['Finding Labels'] = de_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))

all_labels = np.unique(list(chain(*de_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))

all_labels = [x for x in all_labels if len(x) > 0]

print('All Labels ({}): {}'.format(len(all_labels), all_labels))

for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        de_df[c_label] = de_df['Finding Labels'].map(lambda finding: '1' if c_label in finding else '0')

de_df = de_df.drop(columns=['Unnamed: 11','Patient Gender','View Position','Patient Age','Patient ID','Follow-up #','OriginalImagePixelSpacing[x','y]','OriginalImage[Width','Height]'])

NoF_df = de_df[de_df['Finding Labels'] == '' ].sample(30000)


All Labels (14): ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [5]:
# 1/255 adjust the color
datagen = ImageDataGenerator(rescale=1./255 ,validation_split=0.25)
test_datagen = ImageDataGenerator(rescale=1./255)

# Class Mode
Class_Value = 'categorical'

# Target_size for resolution 1:1
TARGET_SIZE = 230

BATCH_SIZE = 32

# To denote a single label to test on
# label = 'Infiltration'
# LabelPicker(label)

# init_gens(BATCH_SIZE ,Class_Value, TARGET_SIZE)


In [6]:
#define Model
model = Sequential()
print(model)
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(TARGET_SIZE,TARGET_SIZE,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:

for label in all_labels:
    print('StartLoop')
    LabelPicker(label)
    
    # Init_Gens will "complete" before its ready to move on. time.sleep(10) is the crude fix.
    init_gens(BATCH_SIZE ,Class_Value, TARGET_SIZE)
    
    # Will need to adjust delay based on the system
    print('Starting Sleep')
    time.sleep(7)
    print('Finishing Sleep')
    
    STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size

    STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size


    model.compile(optimizers.rmsprop(lr=0.0001),
    loss="categorical_crossentropy", metrics=["accuracy"])
    

    model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        workers=16,
                        use_multiprocessing=False,
                        epochs=1) 
    
    
    time.sleep(5)
    print('sleeping part 2')
    model.save(f'Models/{label}.h5')
    print(f' Saving to : SavedModels/{label}.h5')

StartLoop
Starting Sleep
Found 4935 validated image filenames belonging to 2 classes.
Found 6293 validated image filenames belonging to 2 classes.Found 18881 validated image filenames belonging to 2 classes.

Finishing Sleep
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
590/590 [==============================] - 80s 136ms/step - loss: 0.0402 - acc: 0.9976 - val_loss: 0.0488 - val_acc: 0.9970
sleeping part 2
 Saving to : SavedModels/Hernia.h5
StartLoop
Starting Sleep
Found 7111 validated image filenames belonging to 2 classes.
Found 8108 validated image filenames belonging to 2 classes.Found 24327 validated image filenames belonging to 2 classes.

Finishing Sleep
Epoch 1/1
760/760 [==============================] - 98s 129ms/step - loss: 1.7752 - acc: 0.7546 - val_loss: 0.4446 - val_acc: 0.8297
sleeping part 2
 Saving to : SavedModels/Infiltration.h5
StartLoop
Starting Sleep
Found 5334 validated image filenames belonging to 2 classes.
Found 6701 validated image filenames bel

In [7]:
# Save test results to dict for review and CI
test_results_dict = {}
test_size_dict = {}


for label in all_labels:
    print('StartLoop')
    
    LabelPicker(label)
    
    # Init_Gens will "complete" before its ready to move on. time.sleep(10) is the crude fix.
    init_gens(BATCH_SIZE ,Class_Value, TARGET_SIZE)
    print('Starting Sleep')
    time.sleep(7)
    print('Finishing Sleep')

    model = load_model(f'Models/{label}.h5')
    
    STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


    model_test_results = model.evaluate_generator(generator=test_generator,
                        steps=STEP_SIZE_TEST,
                        workers=16,
                        use_multiprocessing=False,
                       )
    print(model_test_results)
    
    test_size = len(test_generator.filenames)
    
    test_results_dict.update({label : model_test_results})
    
    test_size_dict.update({label : test_size})

    
print('EndLoop')

StartLoop
Starting Sleep
Found 4885 validated image filenames belonging to 2 classes.
Found 6359 validated image filenames belonging to 2 classes.
Found 19078 validated image filenames belonging to 2 classes.
Finishing Sleep
Instructions for updating:
Use tf.cast instead.
[0.12194633322513637, 0.9819078947368421]
StartLoop
Starting Sleep
Found 5749 validated image filenames belonging to 2 classes.
Found 6611 validated image filenames belonging to 2 classes.Found 19833 validated image filenames belonging to 2 classes.

Finishing Sleep
[0.621801699631374, 0.8344972067039106]
EndLoop


In [9]:
ResetHard()

Starting Dump
Cuda Closed
Session Cleared


UnboundLocalError: local variable 'model' referenced before assignment

In [8]:
Results = pd.DataFrame(test_results_dict).T
Results.columns = ['loss','test_acc']

test_val = pd.DataFrame(pd.Series(test_size_dict))
test_val.columns = ['sample_size']

final_results = pd.concat([test_val, Results],axis=1)

final_results.to_csv('res/round3.csv')
final_results

,sample_size,loss,test_acc
Pneumonia,4885,0.121946,0.981908
Pneumothorax,5749,0.621802,0.834497
